## 1、loading and preprocessing GT1 data from Yang. et al

In [1]:
import pandas as pd
import numpy as np
import random
from os.path import join
import os
import re
import sys
import time
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from Bio import SeqIO
import warnings
import torch
warnings.filterwarnings("ignore")
CURRENT_DIR = os.getcwd()
print(CURRENT_DIR)

/home/hcyao/Pycharm/GT1-ML


In [2]:
df = pd.read_csv(join(CURRENT_DIR ,"data", "Yang_data", 'Yang_training_set.csv'), sep = ";").copy()
df = df.loc[~pd.isnull(df["ID"])]
df.shape

(59, 75)

In [3]:
enzymes = ['UGT71C1', 'UGT71C2', 'UGT71C3', 'UGT71C4', 'UGT71C5',
       'UGT71B8', 'UGT71B6', 'UGT71B5', 'UGT71B1', 'UGT88A1', 'UGT72E2',
       'UGT72E3', 'UGT72E1', 'UGT72D1', 'UGT72C1', 'UGT72B3', 'UGT72B1',
       'UGT73C3', 'UGT73C4', 'UGT73C5', 'UGT73C6', 'UGT73C1', 'UGT73C7',
       'UGT73B2', 'UGT73B3', 'UGT73B4', 'UGT73B5', 'UGT73B1', 'UGT89B1',
       'UGT89C1', 'UGT84A3', 'UGT84A4', 'UGT84A2', 'UGT84A1', 'UGT84B1',
       'UGT84B2', 'UGT75B1', 'UGT75D1', 'UGT74F1', 'UGT74F2', 'UGT74B1',
       'UGT76E5', 'UGT76E4', 'UGT76E1', 'UGT76E2', 'UGT76E12', 'UGT76E11',
       'UGT76C4', 'UGT85A5', 'UGT85A4', 'UGT78D2', 'UGT78D1', 'VVGT1']
acceptors = df["Name"]
print(len(enzymes))
print(len(acceptors))

53
59


In [4]:
df_activity = pd.DataFrame(columns = ["small molecule", "enzyme", "activity"])

for acceptor in acceptors:
    help_df = df.loc[df["Name"] == acceptor]
    for enzyme in enzymes:
        activity = list(help_df[enzyme])[0]
        if activity <= 1:
            df_activity = df_activity.append({"small molecule" : acceptor, "enzyme" : enzyme, "activity": activity}, ignore_index = True)
            
df_activity.shape

(2847, 3)

In [5]:
name_smile=pd.read_csv('./data/GT1/name_test.csv',sep=',')
df_activity=pd.merge(df_activity,name_smile,how='left',on='small molecule')

In [6]:
InChI_dict = {'Baicalein' : "InChI=1S/C15H10O5/c16-9-6-11(8-4-2-1-3-5-8)20-12-7-10(17)14(18)15(19)13(9)12/h1-7,17-19H",
     'Umbelliferone' : "InChI=1S/C9H6O3/c10-7-3-1-6-2-4-9(11)12-8(6)5-7/h1-5,10H",
     '4-Methyl-umbelliferone' : "InChI=1S/C10H8O3/c1-6-4-10(12)13-9-5-7(11)2-3-8(6)9/h2-5,11H,1H3",
     'Sinapic acid' : "InChI=1S/C11H12O5/c1-15-8-5-7(3-4-10(12)13)6-9(16-2)11(8)14/h3-6,14H,1-2H3,(H,12,13)/b4-3+",
     '4-hydroxyl-benzoic acid' : "InChI=1S/C7H6O3/c8-6-3-1-5(2-4-6)7(9)10/h1-4,8H,(H,9,10)",
    'a-cyano-4-hydroxyl-cinamic acid' : "InChI=1S/C10H7NO3/c11-6-8(10(13)14)5-7-1-3-9(12)4-2-7/h1-5,12H,(H,13,14)/b8-5+",
     '3,4-dichloroaniline' : "InChI=1S/C6H5Cl2N/c7-5-2-1-4(9)3-6(5)8/h1-3H,9H2",
     '3,4-dihydroxylbenzoic acid' : "InChI=1S/C7H6O4/c8-5-2-1-4(7(10)11)3-6(5)9/h1-3,8-9H,(H,10,11)",
    #2,5-dihydroxybenzoic acid
     ' 2,5-dihydroxylbenzoic acid' : "InChI=1S/C7H6O4/c8-4-1-2-6(9)5(3-4)7(10)11/h1-3,8-9H,(H,10,11)",
     ' D-glycerate' : "InChI=1S/C3H6O4/c4-1-2(5)3(6)7/h2,4-5H,1H2,(H,6,7)/p-1/t2-/m1/s1",
    #D-GlcNAc
     ' GlcNAc' : "InChI=1S/C8H15NO6/c1-3(11)9-5-7(13)6(12)4(2-10)15-8(5)14/h4-8,10,12-14H,2H2,1H3,(H,9,11)",
     ' Indole 3-acetate' : "InChI=1S/C10H9NO2/c12-10(13)5-7-6-11-9-4-2-1-3-8(7)9/h1-4,6,11H,5H2,(H,12,13)/p-1",
     ' Gibberellin A3' : "InChI=1S/C19H22O6/c1-9-7-17-8-18(9,24)5-3-10(17)19-6-4-11(20)16(2,15(23)25-19)13(19)12(17)14(21)22/h4,6,10-13,20,24H,1,3,5,7-8H2,2H3,(H,21,22)/t10-,11+,12-,13-,16-,17+,18+,19-/m1/s1",
     ' Gibberellin A4' : "InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)19-6-5-12(20)17(2,16(23)24-19)14(19)13(18)15(21)22/h10-14,20H,1,3-8H2,2H3,(H,21,22)/t10-,11-,12+,13-,14-,17-,18+,19-/m1/s1",
    #(+)-Jasmonic acid 
    ' (?)-Jasmonic acid' : "InChI=1S/C12H18O3/c1-2-3-4-5-10-9(8-12(14)15)6-7-11(10)13/h3-4,9-10H,2,5-8H2,1H3,(H,14,15)/b4-3-/t9-,10-/m0/s1",
     #(+)-cis,trans-Abscisic Acid
     ' (?)- cis, trans Abscisic acid' : "InChI=1S/C15H20O4/c1-10(7-13(17)18)5-6-15(19)11(2)8-12(16)9-14(15,3)4/h5-8,19H,9H2,1-4H3,(H,17,18)/b6-5+,10-7+/t15-/m1/s1",
     ' Kinetin' : "InChI=1S/C10H9N5O/c1-2-7(16-3-1)4-11-9-8-10(13-5-12-8)15-6-14-9/h1-3,5-6H,4H2,(H2,11,12,13,14,15)",
     'Zeatin' : "InChI=1S/C10H13N5O/c1-7(4-16)2-3-11-9-8-10(13-5-12-8)15-6-14-9/h2,5-6,16H,3-4H2,1H3,(H2,11,12,13,14,15)/b7-2+",
     'Luteolin' : "InChI=1S/C15H10O6/c16-8-4-11(19)15-12(20)6-13(21-14(15)5-8)7-1-2-9(17)10(18)3-7/h1-6,16-19H",
     'Quercetin' : "InChI=1S/C15H10O7/c16-7-4-10(19)12-11(5-7)22-15(14(21)13(12)20)6-1-2-8(17)9(18)3-6/h1-5,16-19,21H",
     ' Fisetin' : "InChI=1S/C15H10O6/c16-8-2-3-9-12(6-8)21-15(14(20)13(9)19)7-1-4-10(17)11(18)5-7/h1-6,16-18,20H",
     ' Kaempferol' : "InChI=1S/C15H10O6/c16-8-3-1-7(2-4-8)15-14(20)13(19)12-10(18)5-9(17)6-11(12)21-15/h1-6,16-18,20H",
     'Cinnamic acid' : "InChI=1S/C9H8O2/c10-9(11)7-6-8-4-2-1-3-5-8/h1-7H,(H,10,11)/b7-6+",
     '4-hydroxy cinnamic acid' : "InChI=1S/C9H8O3/c10-8-4-1-7(2-5-8)3-6-9(11)12/h1-6,10H,(H,11,12)/b6-3+",
     '3,4-dihyroxy cinnamic acid' : "InChI=1S/C9H8O4/c10-7-3-1-6(5-8(7)11)2-4-9(12)13/h1-5,10-11H,(H,12,13)/b4-2+",
     '4-hydroxy 3-methoxy cinnamic acid' : "InChI=1S/C10H10O4/c11-6-8-5-7(1-3-9(8)12)2-4-10(13)14/h1-5,11-12H,6H2,(H,13,14)/b4-2+",
     '2-hydroxy cinnamic acid' : "InChI=1S/C9H8O3/c10-8-4-2-1-3-7(8)5-6-9(11)12/h1-6,10H,(H,11,12)",
     '3-hydroxy cinnamic acid' : "InChI=1S/C9H8O3/c10-8-3-1-2-7(6-8)4-5-9(11)12/h1-6,10H,(H,11,12)/b5-4+",
     '7-hydroxy 6-methoxy coumarin (Scopoletin)' : "InChI=1S/C10H8O4/c1-13-9-4-6-2-3-10(12)14-8(6)5-7(9)11/h2-5,11H,1H3",
     '6,7-dihydroxy coumarin (Esculetin)' : "InChI=1S/C9H6O4/c10-6-3-5-1-2-9(12)13-8(5)4-7(6)11/h1-4,10-11H",
     'Threonine' : "InChI=1S/C4H9NO3/c1-2(6)3(5)4(7)8/h2-3,6H,5H2,1H3,(H,7,8)/t2-,3+/m1/s1",
     'Glucose' : "InChI=1S/C6H12O6/c7-1-2-3(8)4(9)5(10)6(11)12-2/h2-11H,1H2/t2-,3-,4+,5-,6?/m1/s1",
     'Dihydrojasmonic acid' : "InChI=1S/C12H20O3/c1-2-3-4-5-10-9(8-12(14)15)6-7-11(10)13/h9-10H,2-8H2,1H3,(H,14,15)",
     'Ser-Phe' : "InChI=1S/C12H16N2O4/c13-9(7-15)11(16)14-10(12(17)18)6-8-4-2-1-3-5-8/h1-5,9-10,15H,6-7,13H2,(H,14,16)(H,17,18)/t9-,10-/m0/s1",
     'Ser-Leu' : "InChI=1S/C9H18N2O4/c1-5(2)3-7(9(14)15)11-8(13)6(10)4-12/h5-7,12H,3-4,10H2,1-2H3,(H,11,13)(H,14,15)/t6-,7-/m0/s1",
     'BocCysThrOMe' : "InChI=1S/C13H24N2O6S/c1-7(16)9(11(18)20-5)15-10(17)8(6-22)14-12(19)21-13(2,3)4/h7-9,16,22H,6H2,1-5H3,(H,14,19)(H,15,17)",
     '1-Thio-S-cyanomethyl-N-acetyl-D-glucosamine' : "InChI=1S/C10H20N2O5S/c1-5(14)12-7-9(16)8(15)6(4-13)17-10(7)18-3-2-11/h6-10,13,15-16H,2-4,11H2,1H3,(H,12,14)",
     'MUGlcNAc' : "InChI=1S/C17H18O8/c1-7-5-12(19)24-11-6-9(3-4-10(7)11)23-17-13(8(2)18)14(20)15(21)16(22)25-17/h3-6,13-17,20-22H,1-2H3",
     'Trans-dihydroquercetin (DHQ)' : "InChI=1S/C15H12O7/c16-7-4-10(19)12-11(5-7)22-15(14(21)13(12)20)6-1-2-8(17)9(18)3-6/h1-5,14-19,21H/t14-,15+/m0/s1",
     '7-hydroxycoumerin 3-carboxylic acid' : "InChI=1S/C10H6O5/c11-6-2-1-5-3-7(9(12)13)10(14)15-8(5)4-6/h1-4,11H,(H,12,13)",
     '7-hydroxycoumerin 4-acetic acid' : "InChI=1S/C11H8O5/c12-7-1-2-8-6(3-10(13)14)4-11(15)16-9(8)5-7/h1-2,4-5,12H,3H2,(H,13,14)",
     'Chloramphenicol' : "InChI=1S/C11H12Cl2N2O5/c12-10(13)11(18)14-8(5-16)9(17)6-1-3-7(4-2-6)15(19)20/h1-4,8-10,16-17H,5H2,(H,14,18)/t8-,9-/m1/s1",
     '?-GlcOBn' : "InChI=1S/C13H18O6/c14-6-9-10(15)11(16)12(17)13(19-9)18-7-8-4-2-1-3-5-8/h1-5,9-17H,6-7H2",
     'a-ManOBn' : "InChI=1S/C13H18O6/c14-6-9-10(15)11(16)12(17)13(19-9)18-7-8-4-2-1-3-5-8/h1-5,9-17H,6-7H2",
     'a-ManOPh' : "InChI=1S/C12H16O6/c13-6-8-9(14)10(15)11(16)12(18-8)17-7-4-2-1-3-5-7/h1-5,8-16H,6H2",
     'a-ManOCH2Bn' : "InChI=1S/C14H20O6/c15-8-10-11(16)12(17)13(18)14(20-10)19-7-6-9-4-2-1-3-5-9/h1-5,10-18H,6-8H2",
     'a-ManOPMP' : "InChI=1S/C13H18O7/c14-5-7-1-3-8(4-2-7)19-13-12(18)11(17)10(16)9(6-15)20-13/h1-4,9-18H,5-6H2",
     'a-ManOBn(pNO2)' : "InChI=1S/C13H19NO8/c15-5-9-10(16)11(17)12(18)13(22-9)21-6-7-1-3-8(4-2-7)14(19)20/h1-4,9-13,15-20H,5-6H2",
     'a-ManOPhF5' : "InChI=1S/C12H11F5O6/c13-3-4(14)6(16)11(7(17)5(3)15)23-12-10(21)9(20)8(19)2(1-18)22-12/h2,8-10,12,18-21H,1H2",
     'a-ManOBnF5' : "InChI=1S/C13H13F5O6/c14-5-3(6(15)8(17)9(18)7(5)16)2-23-13-12(22)11(21)10(20)4(1-19)24-13/h4,10-13,19-22H,1-2H2",
     'ManSTol' : "InChI=1S/C13H18O5S/c1-7-2-4-8(5-3-7)19-13-12(17)11(16)10(15)9(6-14)18-13/h2-5,9-17H,6H2,1H3",
     'Catechin' : "InChI=1S/C15H14O6/c16-8-4-11(18)9-6-13(20)15(21-14(9)5-8)7-1-2-10(17)12(19)3-7/h1-5,13,15-20H,6H2/t13-,15+/m0/s1",
     'Genistein' : "InChI=1S/C15H10O5/c16-9-3-1-8(2-4-9)11-7-20-13-6-10(17)5-12(18)14(13)15(11)19/h1-7,16-18H",
     'N6-Benzyladenine' : "InChI=1S/C12H11N5/c1-2-4-9(5-3-1)6-13-11-10-12(15-7-14-10)17-8-16-11/h1-5,7-8H,6H2,(H2,13,14,15,16,17)",
     'Trans-Zentin-Glucose' : "InChI=1S/C16H23N5O6/c1-8(2-3-17-14-10-15(19-6-18-10)21-7-20-14)5-26-16-13(25)12(24)11(23)9(4-22)27-16/h2,6-7,9,11-13,16,22-25H,3-5H2,1H3,(H2,17,18,19,20,21)/b8-2+",
     'DihydroZeatin' : "InChI=1S/C10H15N5O/c1-7(4-16)2-3-11-9-8-10(13-5-12-8)15-6-14-9/h5-7,16H,2-4H2,1H3,(H2,11,12,13,14,15)",
     'Olomoucine' : "InChI=1S/C15H18N6O/c1-21-10-18-12-13(17-9-11-5-3-2-4-6-11)19-15(16-7-8-22)20-14(12)21/h2-6,10,22H,7-9H2,1H3,(H2,16,17,19,20)",
     'N6-isopentenyladenine' : "InChI=1S/C10H13N5/c1-7(2)3-4-11-9-8-10(13-5-12-8)15-6-14-9/h3,5-6H,4H2,1-2H3,(H2,11,12,13,14,15)",
     'Spectinomycin' : "InChI=1S/C14H24N2O7/c1-5-4-6(17)14(20)13(21-5)22-12-10(19)7(15-2)9(18)8(16-3)11(12)23-14/h5,7-13,15-16,18-20H,4H2,1-3H3/t5-,7-,8+,9+,10+,11-,12-,13+,14+/m1/s1",
     "oleanodmycin" : "InChI=1S/C35H61NO12/c1-16-14-35(15-43-35)32(40)19(4)27(37)18(3)22(7)46-33(41)21(6)31(47-26-13-25(42-11)28(38)23(8)45-26)20(5)30(16)48-34-29(39)24(36(9)10)12-17(2)44-34/h16-31,34,37-39H,12-15H2,1-11H3/t16-,17+,18-,19+,20+,21+,22+,23-,24-,25-,26-,27-,28-,29+,30-,31-,34-,35+/m0/s1",
     "novobiocin" : "InChI=1S/C31H36N2O11/c1-14(2)7-8-16-13-17(9-11-19(16)34)27(37)33-21-22(35)18-10-12-20(15(3)24(18)42-28(21)38)41-29-23(36)25(43-30(32)39)26(40-6)31(4,5)44-29/h7,9-13,23,25-26,29,34-36H,8H2,1-6H3,(H2,32,39)(H,33,37)/t23-,25+,26-,29-/m1/s1",
     "spectinomycin" : "InChI=1S/C14H24N2O7/c1-5-4-6(17)14(20)13(21-5)22-12-10(19)7(15-2)9(18)8(16-3)11(12)23-14/h5,7-13,15-16,18-20H,4H2,1-3H3/t5-,7-,8+,9+,10+,11-,12-,13+,14+/m1/s1",
     "CHAPS" : "InChI=1S/C32H58N2O7S/c1-21(8-11-29(38)33-14-6-15-34(4,5)16-7-17-42(39,40)41)24-9-10-25-30-26(20-28(37)32(24,25)3)31(2)13-12-23(35)18-22(31)19-27(30)36/h21-28,30,35-37H,6-20H2,1-5H3,(H-,33,38,39,40,41)/t21-,22+,23-,24-,25+,26+,27-,28+,30+,31+,32-/m1/s1",
     "solanidine" : "InChI=1S/C27H43NO/c1-16-5-8-23-17(2)25-24(28(23)15-16)14-22-20-7-6-18-13-19(29)9-11-26(18,3)21(20)10-12-27(22,25)4/h6,16-17,19-25,29H,5,7-15H2,1-4H3/t16-,17+,19-,20+,21-,22-,23+,24-,25-,26-,27-/m0/s1",
     "solasodine" : "InChI=1S/C27H43NO2/c1-16-7-12-27(28-15-16)17(2)24-23(30-27)14-22-20-6-5-18-13-19(29)8-10-25(18,3)21(20)9-11-26(22,24)4/h5,16-17,19-24,28-29H,6-15H2,1-4H3/t16-,17+,19+,20-,21+,22+,23+,24+,25+,26+,27-/m1/s1",
     "b-sitosterol"  : "InChI=1S/C29H50O/c1-7-21(19(2)3)9-8-20(4)25-12-13-26-24-11-10-22-18-23(30)14-16-28(22,5)27(24)15-17-29(25,26)6/h10,19-21,23-27,30H,7-9,11-18H2,1-6H3/t20-,21-,23+,24+,25-,26+,27+,28+,29-/m1/s1" }

In [7]:
len(InChI_dict)

66

# 2、calculating ECFP and MACCS

In [35]:
# ECFP 使用2
df_activity["ECFP_1024"] = np.nan
df_activity['SMILES']=""
df_activity['MACCS']= np.nan
small_molecule=df_activity["small molecule"]

for molecule in small_molecule:
    Inchi = InChI_dict[molecule]
    mol = Chem.MolFromInchi(Inchi)
    smile=Chem.MolToSmiles(mol)
    maccs=AllChem.GetMACCSKeysFingerprint(mol).ToBitString()
    ecfp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024).ToBitString()    
    df_activity['SMILES'].loc[df_activity["small molecule"] == molecule]=smile
    df_activity["ECFP_1024"].loc[df_activity["small molecule"] == molecule] = ecfp
    df_activity["MACCS"].loc[df_activity["small molecule"] == molecule] = maccs

In [36]:
df_activity.isnull().sum()

small molecule    0
enzyme            0
activity          0
Smiles            0
ECFP_1024         0
SMILES            0
MACCS             0
dtype: int64

In [37]:
# read fasat
from Bio import SeqIO
df_activity['SEQ']=""
def add_seq(df):
    for fa in SeqIO.parse("./data/Yang_data/Yang_enzymes.fasta", "fasta"):
        if fa.id == df['enzyme']:
            df['SEQ']=str(fa.seq)
    return df

df_activity=df_activity.apply(add_seq,axis=1)

# 3、creating enzyme representations

In [38]:
import torch
df_activity["ESM1b_ts"] = ""

for enzyme in enzymes:
    rep = np.array(torch.load(join(CURRENT_DIR, "data", "Yang_data", "Yang_enzymes",enzyme + ".pt"))["cls_representations"][33])   
    help_df = df_activity.loc[df_activity["enzyme"] ==enzyme]
    for ind in help_df.index:
        df_activity["ESM1b_ts"][ind] = rep

df_activity

,small molecule,enzyme,activity,Smiles,ECFP_1024,SMILES,MACCS,SEQ,ESM1b_ts
0,Baicalein,UGT71C1,0.0,O=C1C=C(C2=CC=CC=C2)OC3=C1C(O)=C(C(O)=C3)O,0000000000000000000000000000000000000000000000...,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,0000000000000000000000000000000000000000000000...,MGKQEDAELVIIPFPFSGHILATIELAKRLISQDNPRIHTITILYW...,"[0.08609772, -0.110735565, -0.29194435, 0.5663..."
1,Baicalein,UGT71C2,0.0,O=C1C=C(C2=CC=CC=C2)OC3=C1C(O)=C(C(O)=C3)O,0000000000000000000000000000000000000000000000...,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,0000000000000000000000000000000000000000000000...,MAKQQEAELIFIPFPIPGHILATIELAKRLISHQPSRIHTITILHW...,"[0.013256749, -0.16644819, -0.34313837, 0.5727..."
2,Baicalein,UGT71C3,1.0,O=C1C=C(C2=CC=CC=C2)OC3=C1C(O)=C(C(O)=C3)O,0000000000000000000000000000000000000000000000...,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,0000000000000000000000000000000000000000000000...,MKAEAEIIFVTYPSPGHLLVSIEFAKSLIKRDDRIHTITILYWALP...,"[0.11302839, -0.028711304, -0.28791505, 0.6181..."
3,Baicalein,UGT71B8,1.0,O=C1C=C(C2=CC=CC=C2)OC3=C1C(O)=C(C(O)=C3)O,0000000000000000000000000000000000000000000000...,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,0000000000000000000000000000000000000000000000...,MNKFALVFVPFPILGHLKSTAEMAKLLVEQETRLSISIIILPLLSG...,"[0.32527673, -0.020017605, -0.23925737, 0.4402..."
4,Baicalein,UGT71B6,1.0,O=C1C=C(C2=CC=CC=C2)OC3=C1C(O)=C(C(O)=C3)O,0000000000000000000000000000000000000000000000...,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,0000000000000000000000000000000000000000000000...,MKIELVFIPSPAISHLMATVEMAEQLVDKNDNLSITVIIISFSSKN...,"[0.32567993, -0.24399604, -0.2138156, 0.499061..."
...,...,...,...,...,...,...,...,...,...
2842,Spectinomycin,UGT76E12,0.0,C[C@@H]1CC([C@@]2(O)[C@@](O[C@]3([H])[C@@H](O)...,0000100000000000000000000000000001001000000000...,CN[C@H]1[C@@H](O)[C@@H](NC)[C@H](O)[C@H]2O[C@@...,0000000000000000000000000000000000000000000000...,MEEKPARRSVVLVPFPAQGHISPMMQLAKTLHLKGFSITVVQTKFN...,"[0.080241196, -0.49778256, -0.100768566, 0.547..."
2843,Spectinomycin,UGT76E11,0.0,C[C@@H]1CC([C@@]2(O)[C@@](O[C@]3([H])[C@@H](O)...,0000100000000000000000000000000001001000000000...,CN[C@H]1[C@@H](O)[C@@H](NC)[C@H](O)[C@H]2O[C@@...,0000000000000000000000000000000000000000000000...,MEEKPAGRRVVLVAVPAQGHISPIMQLAKTLHLKGFSITIAQTKFN...,"[0.026391676, -0.51961315, -0.07424599, 0.4777..."
2844,Spectinomycin,UGT85A5,0.0,C[C@@H]1CC([C@@]2(O)[C@@](O[C@]3([H])[C@@H](O)...,0000100000000000000000000000000001001000000000...,CN[C@H]1[C@@H](O)[C@@H](NC)[C@H](O)[C@H]2O[C@@...,0000000000000000000000000000000000000000000000...,MASHAVTSGQKPHVVCIPFPAQGHINPMLKVAKLLYARGFHVTFVN...,"[-0.02222197, -0.30882728, -0.1440919, 0.38996..."
2845,Spectinomycin,UGT85A4,0.0,C[C@@H]1CC([C@@]2(O)[C@@](O[C@]3([H])[C@@H](O)...,0000100000000000000000000000000001001000000000...,CN[C@H]1[C@@H](O)[C@@H](NC)[C@H](O)[C@H]2O[C@@...,0000000000000000000000000000000000000000000000...,MEQHGGSSSQKPHAMCIPYPAQGHINPMLKLAKLLHARGFHVTFVN...,"[-0.024465147, -0.33846748, -0.10585976, 0.333..."


In [ ]:
# smile 标准化
# from rdkit import Chem
# from rdkit.Chem import AllChem
# def format_smiles(smi):
#     mol=Chem.MolFromSmiles(smi)
#     smi=Chem.MolToSmiles(mol)
#     return smi
# df_activity['Smiles']=df_activity['Smiles'].apply(format_smiles)

In [ ]:
# save data
#save smiles data
df_smiles=pd.DataFrame(columns=['Smiles'])
df_smiles['Smiles']=list(df_activity['Smiles'].value_counts().to_frame().index)
df_smiles.to_csv("./data/gt_smiles.csv",index=False)


# 5、get JTVAE enbeding
JT-VAE 运行命令
python embed.py --vocab ../data/zinc/vocab.txt --hidden 450 --model ../molvae/MPNVAE-h450-L56-d3-beta0.001/model.iter-4 --smiles-list /home/hcyao/Pycharm/GT1-ML/data/gt_smiles.csv --out /home/hcyao/Pycharm/GT1-ML/data/gt-vae-embedding --gpu

In [11]:
import pickle
gt_jtvae=open('./data/JT-VAE/gt-jtvae-embedding.p','rb')
loaded_pickle = pickle.load(gt_jtvae)
# convert dataframe
df_jtvae=pd.DataFrame.from_dict(loaded_pickle).T
df_jtvae=df_jtvae.reset_index().rename(columns={'index':'SMILES'})
df_jtvae['jt-vae']=""
for index, row in df_jtvae.iterrows():
    df_jtvae.at[index,'jt-vae']=np.array(df_jtvae.drop(['SMILES','jt-vae'],axis=1))[index]
df_jtvae=df_jtvae[['SMILES','jt-vae']]
df_activity=pd.merge(df_activity,df_jtvae,left_on='Smiles',right_on='SMILES',how='left')

In [12]:
df_activity.isnull().sum()

small molecule    0
enzyme            0
activity          0
Smiles            0
ECFP_1024         0
SMILES_x          0
MACCS             0
SEQ               0
ESM1b_ts          0
SMILES_y          0
jt-vae            0
dtype: int64

In [ ]:
df_activity.to_pickle(join(CURRENT_DIR, "data", "GT1", "GT1_df_maccs.pkl"))

# 6、transform data into another format(smiles, seq,activity)

In [13]:
df_activity.columns

Index(['small molecule', 'enzyme', 'activity', 'Smiles', 'ECFP_1024',
       'SMILES_x', 'MACCS', 'SEQ', 'ESM1b_ts', 'SMILES_y', 'jt-vae'],
      dtype='object')

In [14]:
df_activity.shape

(2847, 11)

In [ ]:
df_activity['activity']=df_activity['activity'].astype('int')
df_transform=df_activity[[ 'SMILES', 'SEQ','activity']]
df_transform.to_csv('./data/GT1/GT-2847.txt',index=False,sep=" ",header=False)

# 7、验证集数据预处理

### 7.1 berry 按照之前的步骤重新处理一遍

In [39]:
df_Berry = pd.read_csv(join(CURRENT_DIR, "." ,"data", "Yang_data", 'Berry_validation.csv'), sep = ";").copy()
enzymes = ['Lb01/UGT72B10', 'Lb02/UGT74P1', 'Lb04/UGT73A10',
       'Lb06/UGT74N1', 'Lb12/UGT75L2', 'Lb15/UGT85A20', 'Lb17/UGT73Q1',
       'Lb18/UGT73A12', 'Lb19/UGT86A5', 'Lb23/UGT94E2']


df_activity = pd.DataFrame(columns = ["small molecule", "enzyme", "activity"])
acceptors=df_Berry["Acceptor"]

for acceptor in acceptors:
    help_df = df_Berry.loc[df_Berry["Acceptor"] == acceptor]
    for enzyme in enzymes:
        activity = list(help_df[enzyme])[0]
        if activity <= 1:
            df_activity = df_activity.append({"small molecule" : acceptor, "enzyme" : enzyme.split("/")[1], "activity": activity}, ignore_index = True)
            
df_activity.shape

(380, 3)

### add ECFP4 、MACCS and JTVAE

In [40]:
# ECFP 使用2
df_activity["ECFP_1024"] = np.nan
df_activity['SMILES']=""
df_activity['MACCS']= np.nan
small_molecule=df_activity["small molecule"]

for molecule in small_molecule:
    Inchi = InChI_dict[molecule]
    mol = Chem.MolFromInchi(Inchi)
    smile=Chem.MolToSmiles(mol)
    maccs=AllChem.GetMACCSKeysFingerprint(mol).ToBitString()
    ecfp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024).ToBitString()    
    df_activity['SMILES'].loc[df_activity["small molecule"] == molecule]=smile
    df_activity["ECFP_1024"].loc[df_activity["small molecule"] == molecule] = ecfp
    df_activity["MACCS"].loc[df_activity["small molecule"] == molecule] = maccs

### add JTVAE

In [ ]:
df_activity['SMILES'].to_csv('./data/smiles_to_jtvae/berry_smile.csv',index=False,header=None)

### 计算完JTVAE分子指纹之后，再导入

In [18]:
import pickle
gt_jtvae=open('./data/JT-VAE/gt-berry-vae-embedding.p','rb')
loaded_pickle = pickle.load(gt_jtvae)
# convert dataframe
df_jtvae=pd.DataFrame.from_dict(loaded_pickle).T
df_jtvae=df_jtvae.reset_index().rename(columns={'index':'SMILES'})
df_jtvae['jt-vae']=""
for index, row in df_jtvae.iterrows():
    df_jtvae.at[index,'jt-vae']=np.array(df_jtvae.drop(['SMILES','jt-vae'],axis=1))[index]
df_jtvae=df_jtvae[['SMILES','jt-vae']]
df_activity=pd.merge(df_activity,df_jtvae,on='SMILES',how='left')

### calculate enzyme representations

In [19]:
import torch
df_activity["ESM1b_ts"] = ""
pt_files = os.listdir(join(CURRENT_DIR,"data", "Yang_data", "Yang_enzymes"))

for enzyme in enzymes:
    enzyme = enzyme.split("/")[1]
    for file in pt_files:
        if enzyme in file:
            break
    rep = np.array(torch.load(join(CURRENT_DIR ,"data", "Yang_data", "Yang_enzymes",
                               file))["cls_representations"][33])
    help_df = df_activity.loc[df_activity["enzyme"] == enzyme]
    for ind in help_df.index:
        df_activity["ESM1b_ts"][ind] = rep

df_activity

,small molecule,enzyme,activity,ECFP_1024,SMILES,MACCS,jt-vae,ESM1b_ts
0,oleanodmycin,UGT72B10,0,0000000000000000000000000000000001001000000000...,CO[C@H]1C[C@H](O[C@@H]2[C@@H](C)C(=O)O[C@H](C)...,0000000000000000100000100000000000000000000000...,"[-1.7667403, -0.33115336, 0.030015014, 0.14531...","[0.190367, -0.023007678, -0.17562558, 0.215061..."
1,oleanodmycin,UGT74P1,0,0000000000000000000000000000000001001000000000...,CO[C@H]1C[C@H](O[C@@H]2[C@@H](C)C(=O)O[C@H](C)...,0000000000000000100000100000000000000000000000...,"[-1.7667403, -0.33115336, 0.030015014, 0.14531...","[-0.08973988, -0.51375425, -0.1270897, 0.48713..."
2,oleanodmycin,UGT73A10,0,0000000000000000000000000000000001001000000000...,CO[C@H]1C[C@H](O[C@@H]2[C@@H](C)C(=O)O[C@H](C)...,0000000000000000100000100000000000000000000000...,"[-1.7667403, -0.33115336, 0.030015014, 0.14531...","[-0.08658633, -0.027905624, -0.13412698, 0.285..."
3,oleanodmycin,UGT74N1,0,0000000000000000000000000000000001001000000000...,CO[C@H]1C[C@H](O[C@@H]2[C@@H](C)C(=O)O[C@H](C)...,0000000000000000100000100000000000000000000000...,"[-1.7667403, -0.33115336, 0.030015014, 0.14531...","[0.098173656, -0.5818817, -0.100889355, 0.4825..."
4,oleanodmycin,UGT75L2,0,0000000000000000000000000000000001001000000000...,CO[C@H]1C[C@H](O[C@@H]2[C@@H](C)C(=O)O[C@H](C)...,0000000000000000100000100000000000000000000000...,"[-1.7667403, -0.33115336, 0.030015014, 0.14531...","[0.04887647, -0.48787764, 0.08551325, 0.492277..."
...,...,...,...,...,...,...,...,...
375,b-sitosterol,UGT85A20,0,0100000110000000100000000000000001001000000000...,CC[C@H](CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C...,0000000000000000000000000010000000000000000000...,"[0.038009312, -0.46108878, 0.6378203, -0.19282...","[0.08819179, -0.41269886, -0.046921156, 0.6169..."
376,b-sitosterol,UGT73Q1,0,0100000110000000100000000000000001001000000000...,CC[C@H](CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C...,0000000000000000000000000010000000000000000000...,"[0.038009312, -0.46108878, 0.6378203, -0.19282...","[0.12415886, -0.11941293, -0.19394022, 0.27981..."
377,b-sitosterol,UGT73A12,0,0100000110000000100000000000000001001000000000...,CC[C@H](CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C...,0000000000000000000000000010000000000000000000...,"[0.038009312, -0.46108878, 0.6378203, -0.19282...","[-0.09134824, -0.07326901, -0.18867402, 0.3036..."
378,b-sitosterol,UGT86A5,0,0100000110000000100000000000000001001000000000...,CC[C@H](CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C...,0000000000000000000000000010000000000000000000...,"[0.038009312, -0.46108878, 0.6378203, -0.19282...","[0.21679285, -0.45862696, 0.023173656, 0.43411..."


In [ ]:
df_activity.to_pickle(join(CURRENT_DIR, "data", "GT1", "GT1_Berry_df.pkl"))

### 7.1.2 外部测试集保存为txt文件
便于后续输入神经网络的数据集

In [41]:
# read fasat
from Bio import SeqIO
df_activity['SEQ']=""
def add_seq(df):
    for fa in SeqIO.parse("./data/Yang_data/Berry.fasta", "fasta"):
        if fa.id == df['enzyme']:
            df['SEQ']=str(fa.seq)
    return df

df_activity=df_activity.apply(add_seq,axis=1)

In [42]:
df_activity['activity']=df_activity['activity'].astype('int')
df_transform=df_activity[[ 'SMILES', 'SEQ','activity']]

In [45]:
df_transform.to_csv('./data/GT1/Berry-380.txt',index=False,sep=" ",header=False)

### 7.2 Oat  按照上述流程重新处理一遍 
Oat 数据集中一共有266 条数据，38个小分子，7个酶，其中185个零，81个1

In [64]:
df_Oat = pd.read_csv(join(CURRENT_DIR,"data", "Yang_data", 'Oat_validation.csv'), sep = ";").copy()
enzymes = ['As01/UGT84C2', 'As07/UGT85B2', 'As08/UGT74H5',
       'As09/UGT88C4', 'As12/UGT73A5', 'As14/UGT74H6', 'As17/UGT75E3']


df_activity = pd.DataFrame(columns = ["small molecule", "enzyme", "activity"])
acceptors=df_Oat["Acceptor"]

for acceptor in acceptors:
    help_df = df_Oat.loc[df_Oat["Acceptor"] == acceptor]
    for enzyme in enzymes:
        activity = list(help_df[enzyme])[0]
        if activity <= 1:
            df_activity = df_activity.append({"small molecule" : acceptor, "enzyme" : enzyme.split("/")[1], "activity": activity}, ignore_index = True)
df_activity.head()

,small molecule,enzyme,activity
0,oleanodmycin,UGT84C2,0
1,oleanodmycin,UGT85B2,0
2,oleanodmycin,UGT74H5,0
3,oleanodmycin,UGT88C4,0
4,oleanodmycin,UGT73A5,0


### add ECFP4 and MACCS

In [65]:
# ECFP 使用2
df_activity["ECFP_1024"] = np.nan
df_activity['SMILES']=""
df_activity["MACCS"] = np.nan
small_molecule=df_activity["small molecule"]

for molecule in small_molecule:
    Inchi = InChI_dict[molecule]
    mol = Chem.MolFromInchi(Inchi)
    smile=Chem.MolToSmiles(mol)
    maccs=AllChem.GetMACCSKeysFingerprint(mol).ToBitString()
    ecfp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024).ToBitString()    
    df_activity['SMILES'].loc[df_activity["small molecule"] == molecule]=smile
    df_activity["ECFP_1024"].loc[df_activity["small molecule"] == molecule] = ecfp
    df_activity["MACCS"].loc[df_activity["small molecule"] == molecule] = maccs
    
df_activity

,small molecule,enzyme,activity,ECFP_1024,SMILES,MACCS
0,oleanodmycin,UGT84C2,0,0000000000000000000000000000000001001000000000...,CO[C@H]1C[C@H](O[C@@H]2[C@@H](C)C(=O)O[C@H](C)...,0000000000000000100000100000000000000000000000...
1,oleanodmycin,UGT85B2,0,0000000000000000000000000000000001001000000000...,CO[C@H]1C[C@H](O[C@@H]2[C@@H](C)C(=O)O[C@H](C)...,0000000000000000100000100000000000000000000000...
2,oleanodmycin,UGT74H5,0,0000000000000000000000000000000001001000000000...,CO[C@H]1C[C@H](O[C@@H]2[C@@H](C)C(=O)O[C@H](C)...,0000000000000000100000100000000000000000000000...
3,oleanodmycin,UGT88C4,0,0000000000000000000000000000000001001000000000...,CO[C@H]1C[C@H](O[C@@H]2[C@@H](C)C(=O)O[C@H](C)...,0000000000000000100000100000000000000000000000...
4,oleanodmycin,UGT73A5,0,0000000000000000000000000000000001001000000000...,CO[C@H]1C[C@H](O[C@@H]2[C@@H](C)C(=O)O[C@H](C)...,0000000000000000100000100000000000000000000000...
...,...,...,...,...,...,...
261,b-sitosterol,UGT74H5,0,0100000110000000100000000000000001001000000000...,CC[C@H](CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C...,0000000000000000000000000010000000000000000000...
262,b-sitosterol,UGT88C4,0,0100000110000000100000000000000001001000000000...,CC[C@H](CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C...,0000000000000000000000000010000000000000000000...
263,b-sitosterol,UGT73A5,0,0100000110000000100000000000000001001000000000...,CC[C@H](CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C...,0000000000000000000000000010000000000000000000...
264,b-sitosterol,UGT74H6,0,0100000110000000100000000000000001001000000000...,CC[C@H](CC[C@@H](C)[C@H]1CC[C@H]2[C@@H]3CC=C4C...,0000000000000000000000000010000000000000000000...


### add jt-vae

In [ ]:
df_activity['SMILES'].to_csv('./data/smiles_to_jtvae/oat_smile.csv',index=False,header=None)

### finished calculate jt-vae and import

In [ ]:
import pickle
gt_jtvae=open('./data/JT-VAE/gt-oat-vae-embedding.p','rb')
loaded_pickle = pickle.load(gt_jtvae)
# convert dataframe
df_jtvae=pd.DataFrame.from_dict(loaded_pickle).T
df_jtvae=df_jtvae.reset_index().rename(columns={'index':'SMILES'})
df_jtvae['jt-vae']=""
for index, row in df_jtvae.iterrows():
    df_jtvae.at[index,'jt-vae']=np.array(df_jtvae.drop(['SMILES','jt-vae'],axis=1))[index]
df_jtvae=df_jtvae[['SMILES','jt-vae']]
df_activity=pd.merge(df_activity,df_jtvae,on='SMILES',how='left')

### calculate enzyme representations

In [ ]:
import torch
df_activity["ESM1b_ts"] = ""
pt_files = os.listdir(join(CURRENT_DIR,"data", "Yang_data", "Yang_enzymes"))

for enzyme in enzymes:
    enzyme = enzyme.split("/")[1]
    for file in pt_files:
        if enzyme in file:
            break
    rep = np.array(torch.load(join(CURRENT_DIR ,"data", "Yang_data", "Yang_enzymes",
                               file))["cls_representations"][33])
    help_df = df_activity.loc[df_activity["enzyme"] == enzyme]
    for ind in help_df.index:
        df_activity["ESM1b_ts"][ind] = rep

df_activity

In [ ]:
df_activity.to_pickle(join(CURRENT_DIR, "data", "GT1", "GT1_Oat_df.pkl"))

### 7.2.1 外部测试集保存为txt文件
便于后续输入到神经网络的模型

In [74]:
# read fasat
from Bio import SeqIO
df_activity['SEQ']=""
def add_seq(df):
    for fa in SeqIO.parse("./data/Yang_data/Oat.fasta", "fasta"):
        if fa.id == df['enzyme']:
            df['SEQ']=str(fa.seq)
    return df

df_activity=df_activity.apply(add_seq,axis=1)

In [75]:
df_activity['activity']=df_activity['activity'].astype('int')
df_transform=df_activity[[ 'SMILES', 'SEQ','activity']]

In [69]:
df_transform.to_csv('./data/GT1/Oat-266.txt',index=False,sep=" ",header=False)

# 8、Rice _data 应用
主要目的是使用实例应用模型，获得了所有GT1家族的水稻的糖基转移酶，然后预测其天然产物相关的底物，在文献中找到相关的证据支持

## 8.1 提取水稻GT1 的蛋白表示

In [47]:
enzymes = ['LOC_Os10g24090.1', 'LOC_Os06g16000.1', 'LOC_Os09g16090.1', 'LOC_Os09g16030.1', 'LOC_Os04g24110.1', 'LOC_Os09g21170.1', 'LOC_Os09g21160.1',
           'LOC_Os04g20260.1', 'LOC_Os04g42690.1', 'LOC_Os04g20330.1', 'LOC_Os04g20400.2', 'LOC_Os04g20400.1', 'LOC_Os04g20474.4', 'LOC_Os04g20474.3',
           'LOC_Os04g20474.2', 'LOC_Os04g20474.1', 'LOC_Os04g20540.1', 'LOC_Os11g27370.1', 'LOC_Os11g26950.1', 'LOC_Os01g64540.1', 'LOC_Os07g10240.1',
           'LOC_Os07g10160.1', 'LOC_Os07g10190.1', 'LOC_Os07g10230.1', 'LOC_Os03g59030.1', 'LOC_Os02g56010.1', 'LOC_Os02g37690.1', 'LOC_Os03g49550.1',
           'LOC_Os12g37510.1', 'LOC_Os10g03320.1', 'LOC_Os06g11270.1', 'LOC_Os03g49524.1', 'LOC_Os07g47550.1', 'LOC_Os08g15330.1', 'LOC_Os06g09240.1',
           'LOC_Os07g05420.1', 'LOC_Os09g03140.1', 'LOC_Os07g14630.1', 'LOC_Os07g30610.1', 'LOC_Os07g30760.1', 'LOC_Os07g30690.1', 'LOC_Os07g30330.1',
           'LOC_Os07g30620.1', 'LOC_Os07g30469.1', 'LOC_Os07g30369.1', 'LOC_Os11g25454.1', 'LOC_Os07g13940.1', 'LOC_Os07g13810.1', 'LOC_Os07g13660.1',
           'LOC_Os03g60960.1', 'LOC_Os07g13800.1', 'LOC_Os07g13770.1', 'LOC_Os07g13634.1', 'LOC_Os07g42970.1', 'LOC_Os02g51930.1', 'LOC_Os08g31200.1', 
           'LOC_Os04g25490.1', 'LOC_Os04g25440.2', 'LOC_Os04g25440.1', 'LOC_Os04g25800.1', 'LOC_Os04g24850.1', 'LOC_Os04g25980.1', 'LOC_Os04g25370.1', 
           'LOC_Os04g25970.1', 'LOC_Os04g25380.1', 'LOC_Os04g37820.1', 'LOC_Os02g36810.1', 'LOC_Os02g36840.1', 'LOC_Os02g36830.1', 'LOC_Os02g28900.1', 
           'LOC_Os02g51910.1', 'LOC_Os02g51900.1', 'LOC_Os06g11720.1', 'LOC_Os08g07270.1', 'LOC_Os08g07200.1', 'LOC_Os10g17489.1', 'LOC_Os03g55010.1',
           'LOC_Os03g55030.2', 'LOC_Os03g55030.1', 'LOC_Os03g55020.1', 'LOC_Os03g55050.1', 'LOC_Os03g55040.1', 'LOC_Os10g18480.1', 'LOC_Os10g18510.1', 
           'LOC_Os10g18530.1', 'LOC_Os04g44354.3', 'LOC_Os04g44354.1', 'LOC_Os04g44354.4', 'LOC_Os04g44354.2', 'LOC_Os02g42280.1', 'LOC_Os10g30560.1',
           'LOC_Os05g41400.1', 'LOC_Os01g59110.1', 'LOC_Os10g12120.1', 'LOC_Os01g59100.1', 'LOC_Os03g48740.1', 'LOC_Os09g34214.1', 'LOC_Os09g34250.1',
           'LOC_Os09g34270.1', 'LOC_Os09g34230.1', 'LOC_Os04g12980.1', 'LOC_Os04g12690.1', 'LOC_Os04g12960.1', 'LOC_Os04g12710.1', 'LOC_Os04g12970.1',
           'LOC_Os04g12720.1', 'LOC_Os04g12950.1', 'LOC_Os04g12678.1', 'LOC_Os02g09510.1', 'LOC_Os05g47950.1', 'LOC_Os01g49240.1', 'LOC_Os01g49230.1', 
           'LOC_Os06g39330.1', 'LOC_Os02g10880.1', 'LOC_Os06g39270.1', 'LOC_Os11g04860.1', 'LOC_Os11g25990.1', 'LOC_Os05g08750.1', 'LOC_Os01g08440.1',
           'LOC_Os06g18790.1', 'LOC_Os06g18670.1', 'LOC_Os06g18140.1', 'LOC_Os06g18010.1', 'LOC_Os01g50200.1', 'LOC_Os01g53390.1', 'LOC_Os01g53380.1',
           'LOC_Os01g53370.1', 'LOC_Os01g53430.1', 'LOC_Os03g62480.1', 'LOC_Os05g45100.1', 'LOC_Os05g45110.1', 'LOC_Os10g07970.1', 'LOC_Os01g53460.1',
           'LOC_Os05g45090.1', 'LOC_Os05g45080.1', 'LOC_Os05g45200.1', 'LOC_Os01g53330.1', 'LOC_Os06g23560.1', 'LOC_Os01g53350.2', 'LOC_Os01g53350.1', 
           'LOC_Os09g30980.1', 'LOC_Os09g30510.1', 'LOC_Os01g64910.1', 'LOC_Os05g45170.1', 'LOC_Os05g45140.1', 'LOC_Os05g45180.1', 'LOC_Os05g45150.1',
           'LOC_Os01g53420.1', 'LOC_Os07g32630.1', 'LOC_Os07g32620.1', 'LOC_Os07g32020.1', 'LOC_Os07g32060.1', 'LOC_Os07g32010.1', 'LOC_Os07g31960.1',
           'LOC_Os07g37690.1', 'LOC_Os02g14680.1', 'LOC_Os02g14630.1', 'LOC_Os02g14590.1', 'LOC_Os02g14570.1', 'LOC_Os02g14540.1', 'LOC_Os01g43280.1',
           'LOC_Os01g43270.1', 'LOC_Os11g38650.1', 'LOC_Os05g12450.1', 'LOC_Os07g46610.1', 'LOC_Os04g46970.1', 'LOC_Os04g47770.1', 'LOC_Os04g47720.1', 
           'LOC_Os04g46990.1', 'LOC_Os04g46980.1', 'LOC_Os05g42070.1', 'LOC_Os05g42020.1', 'LOC_Os05g42060.1', 'LOC_Os05g42040.1', 'LOC_Os03g59350.1', 
           'LOC_Os03g53350.1', 'LOC_Os03g46400.1', 'LOC_Os05g08510.1', 'LOC_Os01g45140.1', 'LOC_Os01g41450.1', 'LOC_Os01g41430.1', 'LOC_Os01g45110.1', 
           'LOC_Os03g11350.1', 'LOC_Os01g66620.1', 'LOC_Os04g44250.1', 'LOC_Os04g44240.1', 'LOC_Os05g08480.1', 'LOC_Os10g09990.1', 'LOC_Os01g08080.1',
           'LOC_Os02g11700.1', 'LOC_Os06g10860.1', 'LOC_Os08g07170.1', 'LOC_Os08g07180.1', 'LOC_Os06g11710.1', 'LOC_Os07g13780.1', 'LOC_Os02g11680.1', 
           'LOC_Os02g11670.1', 'LOC_Os02g11640.1', 'LOC_Os02g11130.1', 'LOC_Os02g11110.2', 'LOC_Os02g11110.1', 'LOC_Os03g24430.1', 'LOC_Os05g08490.1', 
           'LOC_Os01g08110.1', 'LOC_Os01g08090.1', 'LOC_Os01g08100.1', 'LOC_Os04g55680.1', 'LOC_Os06g17260.1', 'LOC_Os06g17140.1', 'LOC_Os06g17090.1',
           'LOC_Os06g17110.1', 'LOC_Os06g17120.1', 'LOC_Os06g17250.1', 'LOC_Os06g17020.1', 'LOC_Os08g38110.1', 'LOC_Os08g38160.1', 'LOC_Os08g38130.1',
           'LOC_Os06g08830.1', 'LOC_Os06g39080.1', 'LOC_Os06g39070.1', 'LOC_Os04g20360.1', 'LOC_Os03g30934.3', 'LOC_Os03g30934.5', 'LOC_Os03g30934.4',
           'LOC_Os03g30934.2', 'LOC_Os03g30934.1', 'LOC_Os04g12669.1', 'LOC_Os04g04240.1', 'LOC_Os04g25820.1', 'LOC_Os04g24840.1']
acceptors = df["Name"]
print(len(enzymes))
print(len(acceptors))

230
59


In [48]:
print(len(set(enzymes)))

230


In [49]:
df_activity = pd.DataFrame(columns = ["small molecule", "enzyme", "activity"])
for acceptor in acceptors:
    for enzyme in enzymes:
            df_activity = df_activity.append({"small molecule" : acceptor, "enzyme" : enzyme}, ignore_index = True)     
df_activity.shape

(13570, 3)

In [50]:
df_activity

,small molecule,enzyme,activity
0,Baicalein,LOC_Os10g24090.1,NaN
1,Baicalein,LOC_Os06g16000.1,NaN
2,Baicalein,LOC_Os09g16090.1,NaN
3,Baicalein,LOC_Os09g16030.1,NaN
4,Baicalein,LOC_Os04g24110.1,NaN
...,...,...,...
13565,Spectinomycin,LOC_Os03g30934.1,NaN
13566,Spectinomycin,LOC_Os04g12669.1,NaN
13567,Spectinomycin,LOC_Os04g04240.1,NaN
13568,Spectinomycin,LOC_Os04g25820.1,NaN


### 8.2 计算ECFP4分子指纹

In [51]:
# ECFP 使用2
df_activity["ECFP_1024"] = np.nan
df_activity["SMILES"] = np.nan
small_molecule=df_activity["small molecule"]
for molecule in small_molecule:
    Inchi = InChI_dict[molecule]
    mol = Chem.MolFromInchi(Inchi)
    smile=Chem.MolToSmiles(mol)
    ecfp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024).ToBitString()  
    df_activity["ECFP_1024"].loc[df_activity["small molecule"] == molecule] = ecfp
    df_activity['SMILES'].loc[df_activity["small molecule"] == molecule]=smile
    

### 8.3 计算蛋白表示

In [52]:
import torch
df_activity["ESM1b_ts"] = ""

for enzyme in enzymes:
    rep = np.array(torch.load(join(CURRENT_DIR, "data", "Rice_data", "Rice-ESM-1b",enzyme + ".pt"))["cls_representations"][33])   
    help_df = df_activity.loc[df_activity["enzyme"] ==enzyme]
    for ind in help_df.index:
        df_activity["ESM1b_ts"][ind] = rep

df_activity

,small molecule,enzyme,activity,ECFP_1024,SMILES,ESM1b_ts
0,Baicalein,LOC_Os10g24090.1,NaN,0000000000000000000000000000000000000000000000...,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,"[0.15170568, 0.6114924, -0.69178414, 0.9843046..."
1,Baicalein,LOC_Os06g16000.1,NaN,0000000000000000000000000000000000000000000000...,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,"[0.038845424, 0.44078594, 0.25118843, 0.010497..."
2,Baicalein,LOC_Os09g16090.1,NaN,0000000000000000000000000000000000000000000000...,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,"[0.1073294, 0.47225547, 0.31099978, -0.0448946..."
3,Baicalein,LOC_Os09g16030.1,NaN,0000000000000000000000000000000000000000000000...,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,"[0.10608229, 0.48684025, 0.28982338, 0.0151692..."
4,Baicalein,LOC_Os04g24110.1,NaN,0000000000000000000000000000000000000000000000...,O=c1cc(-c2ccccc2)oc2cc(O)c(O)c(O)c12,"[0.056328245, 0.46821052, 0.28641137, -0.00714..."
...,...,...,...,...,...,...
13565,Spectinomycin,LOC_Os03g30934.1,NaN,0000100000000000000000000000000001001000000000...,CN[C@H]1[C@@H](O)[C@@H](NC)[C@H](O)[C@H]2O[C@@...,"[0.020690762, 0.18643089, 0.47412804, 0.157391..."
13566,Spectinomycin,LOC_Os04g12669.1,NaN,0000100000000000000000000000000001001000000000...,CN[C@H]1[C@@H](O)[C@@H](NC)[C@H](O)[C@H]2O[C@@...,"[0.25081506, 0.6468235, -0.7081247, 0.10762866..."
13567,Spectinomycin,LOC_Os04g04240.1,NaN,0000100000000000000000000000000001001000000000...,CN[C@H]1[C@@H](O)[C@@H](NC)[C@H](O)[C@H]2O[C@@...,"[-0.050467268, 0.59640795, 0.24919201, -0.0232..."
13568,Spectinomycin,LOC_Os04g25820.1,NaN,0000100000000000000000000000000001001000000000...,CN[C@H]1[C@@H](O)[C@@H](NC)[C@H](O)[C@H]2O[C@@...,"[-0.15625967, 0.4257539, -0.1140473, 0.0997529..."


In [53]:
len(df_activity['ESM1b_ts'][0])

1280

In [56]:
# 保存成数据格式
df_activity.to_pickle(join(CURRENT_DIR, "data", "GT1", "GT1_Rice_df.pkl"))